In [18]:
!pip install rasterio tifffile numpy opencv-python scikit-learn tqdm -q


In [7]:
!pip install rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 113.3 MB/s eta 0:00:00


In [17]:
import rasterio

path = '/content/patches/mask/Perungudi_Annotation_Mask_2023 (1).tif'

try:
    with rasterio.open(path) as src:
        print("✅ File opened successfully")
        print("Driver:", src.driver)
        print("Width, Height:", src.width, src.height)
        print("Band count:", src.count)
        print("Data type:", src.dtypes)
        print("NoData values:", src.nodatavals)
        print("CRS:", src.crs)
except Exception as e:
    print("❌ Error opening file:", e)


✅ File opened successfully
Driver: GTiff
Width, Height: 335 224
Band count: 1
Data type: ('uint8',)
NoData values: (None,)
CRS: EPSG:4326


In [19]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from tifffile import imread
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split


In [23]:
# Update with your actual paths
sat_image_path = "/content/patches/image/Perungudi_Sentinel2_Composite_2023.tif"   # Sentinel image
mask_image_path = "/content/patches/mask/Perungudi_Annotation_Mask_2023.tif"      # Mask (if available)


In [24]:
def read_tif(path):
    """Safely read GeoTIFF files using rasterio (fallback to tifffile)."""
    try:
        with rasterio.open(path) as src:
            arr = src.read()  # (bands, H, W)
            arr = np.moveaxis(arr, 0, -1)  # → (H, W, bands)
        return arr
    except Exception as e:
        print(f"⚠️ rasterio failed on {path}, retrying with tifffile:", e)
        try:
            arr = imread(path)
            if arr.ndim == 2:
                arr = np.expand_dims(arr, axis=-1)
            return arr
        except Exception as e2:
            raise RuntimeError(f"❌ Could not read {path}: {e2}")


In [26]:
import numpy as np
import rasterio

def safe_read_tif(path):
    """Safely read the GeoTIFF and ignore corrupted tiles."""
    try:
        with rasterio.open(path) as src:
            # Read the image and mask invalid (corrupted) data automatically
            arr = src.read(masked=True)
            arr = np.moveaxis(arr, 0, -1)  # (H, W, bands)
            print("✅ Read successful:", arr.shape)
            return arr
    except Exception as e:
        print("❌ Error reading:", e)
        return None

# Test reading your image (replace with correct path)
sat_img = safe_read_tif("/content/patches/image/Perungudi_Sentinel2_Composite_2023.tif")

# Check if the image was successfully read
if sat_img is not None:
    print("✅ Image read successfully:", sat_img.shape)
else:
    print("❌ Image read failed.")


✅ Read successful: (224, 335, 26)
✅ Image read successfully: (224, 335, 26)


In [27]:
# Normalize each band to [0,1] as usual
if sat_img is not None:
    sat_img = sat_img.astype(np.float32)
    for i in range(sat_img.shape[-1]):
        band = sat_img[..., i]
        min_, max_ = np.min(band), np.max(band)
        sat_img[..., i] = (band - min_) / (max_ - min_ + 1e-6)
    print("✅ Normalization complete.")
else:
    print("❌ Image is empty. Cannot proceed.")


✅ Normalization complete.


In [28]:
PATCH_SIZE = 128
image_patches = []

# Generate patches from the valid data
h, w = sat_img.shape[:2]
for i in range(0, h - PATCH_SIZE, PATCH_SIZE):
    for j in range(0, w - PATCH_SIZE, PATCH_SIZE):
        img_patch = sat_img[i:i+PATCH_SIZE, j:j+PATCH_SIZE]
        if img_patch.shape[:2] == (PATCH_SIZE, PATCH_SIZE):
            image_patches.append(img_patch)

image_patches = np.array(image_patches)
print("✅ Generated patches:", image_patches.shape)


✅ Generated patches: (2, 128, 128, 26)


In [34]:
!pip install tensorflow keras numpy tqdm -q


In [35]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt


In [64]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, Conv2DTranspose, Dropout, BatchNormalization,
    concatenate, Input
)
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2


def align_and_concat(decoder_tensor, encoder_tensor):
    """Resize encoder tensor to match decoder tensor before concatenation."""
    dec_shape = tf.shape(decoder_tensor)
    encoder_resized = tf.image.resize(
        encoder_tensor,
        size=(dec_shape[1], dec_shape[2]),
        method='bilinear'
    )
    return concatenate([decoder_tensor, encoder_resized], axis=-1)


def irunet(input_shape=(256, 256, 3), num_classes=3):
    # Encoder: InceptionResNetV2 pretrained on ImageNet
    base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)

    # Encoder feature blocks for skip connections - Using identified layer names from cell bb41bcd1
    # Based on the output of cell bb41bcd1, selecting layers that seem appropriate for skip connections
    # based on their position in the network and typical UNet architecture.
    # These names might need adjustment based on the specific InceptionResNetV2 implementation.
    skips = [
        base_model.get_layer('activation_611').output,  # High resolution (after initial conv)
        base_model.get_layer('activation_613').output,  # Mid-high resolution (after first max pooling)
        base_model.get_layer('activation_615').output,  # Mid resolution (after second max pooling)
        base_model.get_layer('activation_681').output,  # Low resolution (before mixed_7a)
        base_model.get_layer('activation_771').output  # Deepest (before block8_1)
    ]

    enc0, enc1, enc2, enc3, enc4 = skips


    # Bridge Block (used as bottleneck, adapted from UNet style)
    bridge = Conv2D(256, (3, 3), activation='relu', padding='same')(enc4)
    bridge = Dropout(0.5)(bridge)
    bridge = Conv2D(256, (3, 3), activation='relu', padding='same')(bridge)

    # Decoder
    # Get shapes of skip connections for target_shape in Conv2DTranspose
    enc3_shape = tf.shape(enc3)
    dec4 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same', target_shape=(enc3_shape[0], enc3_shape[1], enc3_shape[2], 256))(bridge)
    dec4 = concatenate([dec4, enc3])
    dec4 = Conv2D(256, (3, 3), activation='relu', padding='same')(dec4)
    dec4 = BatchNormalization()(dec4)
    dec4 = Conv2D(256, (3, 3), activation='relu', padding='same')(dec4)
    dec4 = BatchNormalization()(dec4)

    enc2_shape = tf.shape(enc2)
    dec3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', target_shape=(enc2_shape[0], enc2_shape[1], enc2_shape[2], 128))(dec4)
    dec3 = concatenate([dec3, enc2])
    dec3 = Conv2D(128, (3, 3), activation='relu', padding='same')(dec3)
    dec3 = BatchNormalization()(dec3)
    dec3 = Conv2D(128, (3, 3), activation='relu', padding='same')(dec3)
    dec3 = BatchNormalization()(dec3)

    enc1_shape = tf.shape(enc1)
    dec2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', target_shape=(enc1_shape[0], enc1_shape[1], enc1_shape[2], 64))(dec3)
    dec2 = concatenate([dec2, enc1])
    dec2 = Conv2D(64, (3, 3), activation='relu', padding='same')(dec2)
    dec2 = BatchNormalization()(dec2)
    dec2 = Conv2D(64, (3, 3), activation='relu', padding='same')(dec2)
    dec2 = BatchNormalization()(dec2)

    enc0_shape = tf.shape(enc0)
    dec1 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same', target_shape=(enc0_shape[0], enc0_shape[1], enc0_shape[2], 32))(dec2)
    dec1 = concatenate([dec1, enc0])
    dec1 = Conv2D(32, (3, 3), activation='relu', padding='same')(dec1)
    dec1 = BatchNormalization()(dec1)
    dec1 = Conv2D(32, (3, 3), activation='relu', padding='same')(dec1)
    dec1 = BatchNormalization()(dec1)


    # Output Block
    output = Conv2D(num_classes, (1, 1), activation='softmax', padding='same')(dec1)
    model = Model(inputs=base_model.input, outputs=output)

    return model

In [65]:
input_shape = (256, 256, 3)
num_classes = 3

model = irunet(input_shape, num_classes)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=num_classes)]
)

model.summary()


ValueError: No such layer: activation_611. Existing layers are: ['input_layer_16', 'conv2d_3260', 'batch_normalization_3252', 'activation_3248', 'conv2d_3261', 'batch_normalization_3253', 'activation_3249', 'conv2d_3262', 'batch_normalization_3254', 'activation_3250', 'max_pooling2d_64', 'conv2d_3263', 'batch_normalization_3255', 'activation_3251', 'conv2d_3264', 'batch_normalization_3256', 'activation_3252', 'max_pooling2d_65', 'conv2d_3268', 'batch_normalization_3260', 'activation_3256', 'conv2d_3266', 'conv2d_3269', 'batch_normalization_3258', 'batch_normalization_3261', 'activation_3254', 'activation_3257', 'average_pooling2d_16', 'conv2d_3265', 'conv2d_3267', 'conv2d_3270', 'conv2d_3271', 'batch_normalization_3257', 'batch_normalization_3259', 'batch_normalization_3262', 'batch_normalization_3263', 'activation_3253', 'activation_3255', 'activation_3258', 'activation_3259', 'mixed_5b', 'conv2d_3275', 'batch_normalization_3267', 'activation_3263', 'conv2d_3273', 'conv2d_3276', 'batc

In [43]:
# Instantiate the InceptionResNetV2 model with a dummy input shape
base_model = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

# Print the names of all layers in the model
for layer in base_model.layers:
    print(layer.name)

input_layer_3
conv2d_611
batch_normalization_609
activation_609
conv2d_612
batch_normalization_610
activation_610
conv2d_613
batch_normalization_611
activation_611
max_pooling2d_12
conv2d_614
batch_normalization_612
activation_612
conv2d_615
batch_normalization_613
activation_613
max_pooling2d_13
conv2d_619
batch_normalization_617
activation_617
conv2d_617
conv2d_620
batch_normalization_615
batch_normalization_618
activation_615
activation_618
average_pooling2d_3
conv2d_616
conv2d_618
conv2d_621
conv2d_622
batch_normalization_614
batch_normalization_616
batch_normalization_619
batch_normalization_620
activation_614
activation_616
activation_619
activation_620
mixed_5b
conv2d_626
batch_normalization_624
activation_624
conv2d_624
conv2d_627
batch_normalization_622
batch_normalization_625
activation_622
activation_625
conv2d_623
conv2d_625
conv2d_628
batch_normalization_621
batch_normalization_623
batch_normalization_626
activation_621
activation_623
activation_626
block35_1_mixed
block35

In [38]:
from tensorflow.keras.utils import to_categorical

# Assume you already have your training and validation data in X_train, Y_train, X_val, Y_val
# Convert the labels to categorical (one-hot encoding)
Y_train_cat = to_categorical(Y_train, num_classes=num_classes)
Y_val_cat = to_categorical(Y_val, num_classes=num_classes)

# Check the shape of the labels to ensure they are correctly one-hot encoded
print("Y_train_cat shape:", Y_train_cat.shape)
print("Y_val_cat shape:", Y_val_cat.shape)


NameError: name 'Y_train' is not defined